<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Example</a></span></li></ul></div>

# Mediator

## Intro

The pattern defines an object that encapsulates how a set of objects interact. It lets you vary their interaction independently. It takes care of comms between a group of objects. Just like an IRL mediator.

All comms must go through the mediator. This is so that it can track and also record all communication between participants. The participants are also aware of the mediator and should be aware that comms go through it.

The pattern is used to reduce communication complexity between multiple class and or objects

A good example of a mediator is a radio traffic controller. It provides a route for all communications to be channeled across and directed to the correct place. it is also able to adjudicate on how communication should be transmitted

The mediator helps to solve an object structure that has become complicated. It allows for correct communication to occur between a group of deeply interconnected objects. of course, the objects will be decoupled if good principles have been followed. But, once a certain level of complexity is reached, the system itself should be in control of how comms are managed. Hence this pattern.

Advantages of the pattern are:
* Increases reusability of objects supported inside the pattern
* Simplifies the maintenance of the system by centralising control logic
* Reduces the variety of messages being sent between objects

N.B. this pattern does not exist to produce lose coupling. It exists when the graph of interconnect objects becomes to dense to enable any meaningful change without breaking stuff. Use this pattern to get a handle on comms between cooperating bits in a complex system

## Implementation

In [4]:
abstract class User {
    protected ChatMediator mediator;
    protected String name;
    
    public User(ChatMediator med, String name)
    {
        this.mediator = med;
        this.name = name;
    }
    
    abstract void send(String msg);
    abstract void receive(String msg);
} 

In [5]:
interface ChatMediator {
     void sendMessage(String msg, User user);
     void addUser(User user);
 }

In [6]:
class ChatMediatorImpl implements ChatMediator {
    private List<User> users;
    
    public ChatMediatorImpl()
    {
        this.users = new ArrayList<User>();
    }
    
    public void addUser(User usr)
    {
        this.users.add(usr);
    }
    
    public void sendMessage(String msg, User usr)
    {
        for (User user : this.users) {
            if (user != usr) {
                user.receive(msg);
            }
        }
    }
}

In [16]:
class UserImpl extends User {
    public UserImpl(ChatMediator med, String name)
    {
        super(med, name);
    }
    
    public void send(String msg)
    {
        System.out.println(this.name + " said " + msg);
        mediator.sendMessage(msg, this);
    }
    
    public void receive(String msg)
    {
        System.out.println(this.name + " received " + msg);
    }
}

In [17]:
ChatMediator myMed = new ChatMediatorImpl();

In [18]:
User user1 = new UserImpl(myMed, "Edward");
User user2 = new UserImpl(myMed, "Thomas");
User user3 = new UserImpl(myMed, "Gary");

In [19]:
myMed.addUser(user1);
myMed.addUser(user2);
myMed.addUser(user3);

In [20]:
user1.send("This is my message");

Edward said This is my message
Thomas received This is my message
Gary received This is my message


## Example

In [16]:
abstract class Buyer {
    float bid;
    String name;
    
    public Buyer(String name, float bid)
    {
        this.bid = bid;
        this.name = name;
    }
    
    public void bid(float bidAmt)
    {
        this.bid += bidAmt;
    }
    
    public void auctionOver()
    {
        System.out.println("I lost the auction :(");
    }
}

In [17]:
interface AuctionMediator {
    Buyer findHighestBidder();
}

In [18]:
class ConcBuyer extends Buyer {
    public ConcBuyer(String name, float bid)
    {
        super(name, bid);
    }
}

In [30]:
class AuctionMediatorImpl implements AuctionMediator {
    List<Buyer> list = new ArrayList<>();
    
    public Buyer findHighestBidder()
    {
        Buyer highestBidder = new ConcBuyer("No buyer yet", 0f);
        for (Buyer buyer : list) {
            if (buyer.bid > highestBidder.bid) {
                highestBidder = buyer;
            }
        }
        
        return highestBidder;
    }
    
    public void addBuyer(Buyer b)
    {
        this.list.add(b);
    }
    
    public void closeAuction()
    {
        Buyer winningBuyer = findHighestBidder();
        for (Buyer buyer : list) {
            if (winningBuyer != buyer) {
                buyer.auctionOver();
            } 
        }
        
        System.out.println(winningBuyer.name + " won the auction with a bid of " + winningBuyer.bid);
    }
}

In [31]:
Buyer b1 = new ConcBuyer("Tom", 32.4f);
Buyer b2 = new ConcBuyer("Ang", 23.4f);
Buyer b3 = new ConcBuyer("Mikey", 89.4f);

In [32]:
AuctionMediatorImpl auction = new AuctionMediatorImpl();

In [33]:
auction.addBuyer(b1);
auction.addBuyer(b2);
auction.addBuyer(b3);

In [35]:
auction.findHighestBidder().name;

Mikey

In [36]:
b2.bid(400.43f);

In [37]:
auction.findHighestBidder().name;

Ang

In [38]:
auction.closeAuction();

I lost the auction :(
I lost the auction :(
Ang won the auction with a bid of 423.83
